In [1]:
import numpy as np
import pandas as pd

In [6]:
opisy_ofert = []

In [5]:
opisy_ofert.append(

['Mieszkanie 2 pokojowe w centrum miasta']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

etykiety = [1, 0, 1, 0, 1]  # 1 - prawdziwa oferta, 0 - fałszywa oferta

# Tworzenie wektora cech
vectorizer = CountVectorizer()
cechy = vectorizer.fit_transform(opisy_ofert)

# Konwersja wektora cech do ramki danych
cechy_df = pd.DataFrame(cechy.toarray(), columns=vectorizer.get_feature_names_out())

# Dodawanie etykiet
cechy_df['label'] = etykiety

# Wyświetlanie ramki danych
print(cechy_df)


   dla  doświadczonego  dynamicznym  dziale  endowymi  etat  front  it   
0    0               1            0       0         0     0      0   0  \
1    0               0            1       0         0     0      0   0   
2    1               0            0       1         0     0      0   1   
3    0               0            0       0         1     0      1   0   
4    0               0            0       0         0     1      0   0   

   juniora  na  ...  pracę  programisty  python  szukamy  umiejętnościami   
0        0   0  ...      0            1       1        1                0  \
1        0   0  ...      0            0       0        0                0   
2        1   0  ...      0            0       0        0                0   
3        0   0  ...      0            0       0        0                1   
4        0   1  ...      1            0       0        0                0   

   umowa  zatrudnimy  zdalna  środowisku  label  
0      0           0       0           0  